In [2]:
import numpy as np
import pandas as pd
import librosa as lib
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix,multilabel_confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import coo_matrix


## Importar los rttm y convertirlos en un df

In [3]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm

# ============================================================
# Config
# ============================================================

BASE_DIR = Path("transcripciones-segunda-vuelta/random")
OUT_DIR = Path("outputs/transcripciones-segunda-vuelta/random")
OUT_DIR.mkdir(parents=True, exist_ok=True)

STEP = 0.01  # tu resolución temporal

# ============================================================
# Helpers de lectura RTTM (tus funciones)
# ============================================================
def read_rttm(file_path):
    columns = ['Type', 'File ID', 'Channel', 'Start Time', 'Duration', 'Ortho', 'Ortho1', 'SType', 'Conf']
    df = pd.read_csv(file_path, delim_whitespace=True, header=None, names=columns)
    df['Start Time'] = df['Start Time'].astype(float)
    df['Duration'] = df['Duration'].astype(float)
    df['Conf'] = df['Conf'].astype(float)
    return df

def read_rttm_diar(file_path):
    columns = ['Type', 'File ID', 'Channel', 'Start Time', 'Duration', 'Ortho1', 'Ortho2', 'SType', 'Name1', 'Name2']
    df = pd.read_csv(file_path, delim_whitespace=True, header=None, names=columns)
    df['Start Time'] = df['Start Time'].astype(float)
    df['Duration'] = df['Duration'].astype(float)
    return df

def merge_intervals(iv):
    if not iv:
        return iv
    iv_sorted = sorted(iv, key=lambda x: x[0])
    merged = [iv_sorted[0]]
    for s, e in iv_sorted[1:]:
        last_s, last_e = merged[-1]
        if s <= last_e:  # solapado o contiguo
            merged[-1] = (last_s, max(last_e, e))
        else:
            merged.append((s, e))
    return merged

# ============================================================
# Preparar el mapeo de archivos: base -> {elan, diar}
#   - "elan"  = *.rttm sin sufijo "-diarization"
#   - "diar"  = *-diarization.rttm
# ============================================================
pairs = {}
for f in BASE_DIR.glob("*.rttm"):
    fn = f.name
    if fn.endswith("-diarization.rttm"):
        base = fn[:-len("-diarization.rttm")]  # quita el sufijo
        pairs.setdefault(base, {})["diar"] = f
    else:
        base = fn[:-len(".rttm")]             # base sin extensión
        pairs.setdefault(base, {})["elan"] = f

# ============================================================
# Función que procesa un par (elan + diar) y devuelve df_results
# ============================================================
def process_pair(base_name, elan_path: Path, diar_path: Path, step=STEP):
    # ---------------- Leer
    df_rttm = read_rttm(elan_path)
    df_rttm_diar = read_rttm_diar(diar_path)

    # ---------------- Mapear SType -> STypeNew (Elan)
    conditions = [
        df_rttm["SType"] == "CHI",
        df_rttm["SType"].str.startswith("F"),
        df_rttm["SType"].str.startswith("M"),
        df_rttm["SType"].str.startswith("UC"),
        df_rttm["SType"].str.startswith("E")
    ]
    choices = ["KCHI", "FEM", "MAL", "OCH", "ELE"]
    df_rttm["STypeNew"] = np.select(conditions, choices, default=df_rttm["SType"])

    # ---------------- STypeNew para diar (tal como en tu código)
    df_rttm_diar["STypeNew"] = df_rttm_diar["SType"]

    # ---------------- Tipificar 'Type' y columnas mínimas
    df_rttm['Type'] = 'Elan'
    df_rttm_diar['Type'] = 'Diar'

    df_rttm = df_rttm[['Type', 'File ID', 'Start Time', 'Duration', 'STypeNew']]
    df_rttm_diar = df_rttm_diar[['Type', 'File ID', 'Start Time', 'Duration', 'STypeNew']]

    # ---------------- Concatenar ambos
    df_both = pd.concat([df_rttm, df_rttm_diar], ignore_index=True)
    df_both = df_both.dropna(subset=['STypeNew'])

    # ---------------- End Time (acelera filtros)
    if "End Time" not in df_both.columns:
        df_both["End Time"] = df_both["Start Time"] + df_both["Duration"]

    # ---------------- Filtrar "code" en Elan para construir intervalos
    #     (si no hay "code", se procesará 0 pasos)
    df_rttm["Start Time"] = pd.to_numeric(df_rttm["Start Time"], errors="coerce")
    df_rttm["Duration"] = pd.to_numeric(df_rttm["Duration"], errors="coerce")

    df_code = (
        df_rttm.loc[df_rttm["STypeNew"].eq("code"), ["Start Time", "Duration"]]
               .dropna()
               .query("`Duration` > 0")
               .sort_values("Start Time", kind="stable")
    )

    intervals = list(
        df_code.apply(lambda r: (float(r["Start Time"]), float(r["Start Time"] + r["Duration"])), axis=1)
    )
    intervals = merge_intervals(intervals)

    # ---------------- Si no hay intervalos "code", devolvemos vacío con metadata
    if not intervals:
        return pd.DataFrame({
            'Elan_KCHI': [], 'Elan_OCH': [], 'Elan_FEM': [], 'Elan_MAL': [], 'Elan_ELE': [],
            'Diar_KCHI': [], 'Diar_OCH': [], 'Diar_FEM': [], 'Diar_MAL': [], 'Diar_SPEECH': []
        })

    # ---------------- Inicializar resultados
    results = {
        'Elan_KCHI': [], 'Elan_OCH': [], 'Elan_FEM': [], 'Elan_MAL': [], 'Elan_ELE': [],
        'Diar_KCHI': [], 'Diar_OCH': [], 'Diar_FEM': [], 'Diar_MAL': [], 'Diar_SPEECH': []
    }

    # ---------------- Calcular total de pasos
    total_steps = 0
    for s, e in intervals:
        if e > s:
            total_steps += int(np.ceil((e - s) / step))

    # ---------------- Iterar con barra de progreso
    with tqdm(total=total_steps, desc=f"{base_name}: Processing intervals ({len(intervals)} blocks)") as pbar:
        for s, e in intervals:
            if e <= s:
                continue
            # Generamos una vista para reducir scans:
            # (Esto aún es simple; si querés más performance se puede indexar por tiempo)
            for i in np.arange(s, e, step):
                for key in results.keys():
                    type_name, stype_new = key.split('_', 1)  # 'Elan' / 'Diar', 'KCHI'/...
                    has_any = (
                        df_both[
                            (df_both['Start Time'] <= i) &
                            (df_both['End Time'] > i) &
                            (df_both['STypeNew'] == stype_new) &
                            (df_both['Type'] == type_name)
                        ].shape[0] > 0
                    )
                    results[key].append(int(has_any))
                pbar.update(1)

    df_results = pd.DataFrame(results)
    return df_results

# ============================================================
# Loop general sobre todos los pares encontrados
# ============================================================
all_outputs = []  # para concatenar si querés un único CSV global

print(f"Se encontraron {len(pairs)} posibles bases en {BASE_DIR.resolve()}\n")

for base_name, d in pairs.items():
    if "elan" not in d or "diar" not in d:
        # Aviso si falta alguno de los dos archivos
        faltante = "elan" if "elan" not in d else "diar"
        print(f"[AVISO] Salteando '{base_name}': falta archivo {faltante}.")
        continue

    elan_path = d["elan"]
    diar_path = d["diar"]

    print(f"Procesando: {base_name}")
    df_results = process_pair(base_name, elan_path, diar_path, step=STEP)

    # Guardar cada resultado individual
    out_file = OUT_DIR / f"{base_name}_results.csv"
    df_results.to_csv(out_file, index=False)
    print(f" -> Guardado: {out_file}")

    # (Opcional) agregar metadata para concatenado global
    if not df_results.empty:
        df_results["name"] = base_name
        all_outputs.append(df_results)

# ============================================================
# (Opcional) Un único CSV global con todos los archivos
# ============================================================
if all_outputs:
    df_all = pd.concat(all_outputs, ignore_index=True)
#     df_all.to_csv(OUT_DIR / "_all_results.csv", index=False)
    print(f"\nArchivo global: {OUT_DIR / '_all_results.csv'}")
else:
    print("\nNo hubo resultados (posiblemente no hay intervalos 'code' en Elan).")


Se encontraron 3 posibles bases en C:\Users\pablo\Documents\CIIPME\Pablo\tesis_pablo\transcripciones-segunda-vuelta\random

[AVISO] Salteando 'bautista-a1-nsb': falta archivo elan.
[AVISO] Salteando 'bautistad-a1-nsb': falta archivo diar.
Procesando: camilob-a1-nsm


camilob-a1-nsm: Processing intervals (15 blocks): 100%|██████████| 180000/180000 [41:01<00:00, 73.11it/s] 


 -> Guardado: outputs\transcripciones-segunda-vuelta\random\camilob-a1-nsm_results.csv

Archivo global: outputs\transcripciones-segunda-vuelta\random\_all_results.csv


['CHI', 'FA1', 'code', 'MC1', 'MA1', 'EE1', 'MC2', 'FC1', 'FA2', 'FA3', 'FA4', 'FA5', 'FA6', 'MC3', 'FA7', 'MA2', 'MC4', 'UC1']


,Type,File ID,Channel,Start Time,Duration,Ortho,Ortho1,SType,Conf,STypeNew
0,SPEAKER,biancak-a2-nsb,1,123.88,0.65,NaN,NaN,CHI,1.0,KCHI
1,SPEAKER,biancak-a2-nsb,1,140.27,4.26,NaN,NaN,CHI,1.0,KCHI
2,SPEAKER,biancak-a2-nsb,1,145.02,1.08,NaN,NaN,CHI,1.0,KCHI
3,SPEAKER,biancak-a2-nsb,1,149.05,0.64,NaN,NaN,CHI,1.0,KCHI
4,SPEAKER,biancak-a2-nsb,1,151.03,0.98,NaN,NaN,CHI,1.0,KCHI
...,...,...,...,...,...,...,...,...,...,...
841,SPEAKER,biancak-a2-nsb,1,11512.75,0.57,NaN,NaN,MC4,1.0,MAL
842,SPEAKER,biancak-a2-nsb,1,12639.02,0.39,NaN,NaN,MC4,1.0,MAL
843,SPEAKER,biancak-a2-nsb,1,13297.77,0.39,NaN,NaN,MC4,1.0,MAL
844,SPEAKER,biancak-a2-nsb,1,13308.74,1.21,NaN,NaN,MC4,1.0,MAL


biancak-a2-nsb
Contenido de df_both:
      Type                     File ID  Start Time  Duration STypeNew  \
0     Elan              biancak-a2-nsb     123.880     0.650     KCHI   
1     Elan              biancak-a2-nsb     140.270     4.260     KCHI   
2     Elan              biancak-a2-nsb     145.020     1.080     KCHI   
3     Elan              biancak-a2-nsb     149.050     0.640     KCHI   
4     Elan              biancak-a2-nsb     151.030     0.980     KCHI   
...    ...                         ...         ...       ...      ...   
6218  Diar  biancak-a2-nsb.diarization   14631.909     1.681   SPEECH   
6219  Diar  biancak-a2-nsb.diarization   14632.010     1.660      CHI   
6220  Diar  biancak-a2-nsb.diarization   14634.290     1.958      CHI   
6221  Diar  biancak-a2-nsb.diarization   14634.493     1.799   SPEECH   
6222  Diar  biancak-a2-nsb.diarization   14635.492     0.597     KCHI   

      Channel  Ortho1  Ortho2   SType  Name1  Name2  
0         NaN     NaN     NaN   

C:\Users\pablo\AppData\Local\Temp\ipykernel_30828\3335347668.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rttm["Start Time"] = pd.to_numeric(df_rttm["Start Time"], errors="coerce")
C:\Users\pablo\AppData\Local\Temp\ipykernel_30828\3335347668.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rttm["Duration"]  = pd.to_numeric(df_rttm["Duration"],  errors="coerce")


Processing intervals (15 blocks):  29%|██▉       | 51786/180000 [14:27<36:13, 58.99it/s]  

In [ ]:
df_results.to_csv(f"resultados/{name}/df_results_{name}.csv", index=False)


In [ ]:
df_resultados= pd.read_csv(f"resultados/{name}/df_results_{name}.csv")
df_resultados

In [ ]:
# Definir las etiquetas
labels = ['KCHI', 'OCH', 'FEM', 'MAL']

# Extraer las etiquetas verdaderas y predichas
y_true = df_resultados[['Elan_KCHI', 'Elan_OCH', 'Elan_FEM', 'Elan_MAL']].values
y_pred = df_resultados[['Diar_KCHI', 'Diar_OCH', 'Diar_FEM', 'Diar_MAL']].values

# Calcular la matriz de confusión multilabel
conf_matrix = multilabel_confusion_matrix(y_true, y_pred)

# Mostrar la matriz de confusión para cada etiqueta
for i, label in enumerate(labels):
    print(f"Matriz de confusión para la etiqueta {label}:")
    print(conf_matrix[i])
    print()

# Opcionalmente, puedes imprimir un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_true, y_pred, target_names=labels))


In [ ]:
# Conversión de etiquetas multilabel a etiquetas de clase única (mismo código que antes)
def multilabel_to_singlelabel(y_multilabel):
    single_labels = []
    for row in y_multilabel:
        indices = np.where(row == 1)[0]
        if len(indices) == 0:
            single_labels.append(-1)  # Sin etiqueta
        else:
            # Si hay múltiples etiquetas, puedes decidir cómo manejarlas.
            # Aquí, tomamos la primera etiqueta encontrada.
            single_labels.append(indices[0])
    return np.array(single_labels)

y_true = multilabel_to_singlelabel(y_true)
y_pred = multilabel_to_singlelabel(y_pred)

# Filtrar las instancias sin etiqueta
valid_indices = y_true != -1
y_true = y_true[valid_indices]
y_pred = y_pred[valid_indices]

# Calcular la matriz de confusión normalizada
cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)), normalize='true')

# Crear un DataFrame para la matriz de confusión
cm_df = pd.DataFrame(cm, index=labels, columns=labels)

# Crear las anotaciones con porcentajes
annot = cm_df.applymap(lambda x: '{0:.1f}%'.format(x*100))


# Crear el gráfico de la matriz de confusión con porcentajes
plt.figure(figsize=(8,6))
sns.heatmap(cm_df, annot=annot, fmt='', cmap='Blues', cbar_kws={'format': '%.0f%%'})
plt.title('Matriz de Confusión Normalizada entre Elan y Diar')
plt.ylabel('Etiquetas Verdaderas (Elan)')
plt.xlabel('Etiquetas Predichas (Diar)')

# Guardar la figura como un archivo PNG
plt.savefig(f'resultados/{name}/matriz_confusion_{name}.png', dpi=300, bbox_inches='tight')

plt.show()


In [ ]:
# Cálculo de precisión, recall y F1-score
types = ['KCHI', 'OCH', 'FEM', 'MAL']

# Inicializar listas para almacenar las métricas
metrics = {
    'Tipo': [],
    'Precisión': [],
    'Recall': [],
    'F1-score': []
}

print("\nCalculando métricas de precisión, recall y F1-score:")
for t in tqdm(types, desc="Procesando tipos"):
    y_true = df_results['Elan_' + t]
    y_pred = df_results['Diar_' + t]
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    # Almacenar las métricas
    metrics['Tipo'].append(t)
    metrics['Precisión'].append(precision)
    metrics['Recall'].append(recall)
    metrics['F1-score'].append(f1)
    
    # Imprimir los resultados
    print(f"\nResultados para el tipo {t}:")
    print(f"Precisión: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")

# Crear DataFrame de métricas
df_metrics = pd.DataFrame(metrics)

In [ ]:
# Generar gráficos
sns.set(style="whitegrid")

# Gráfico de barras para Precisión
plt.figure(figsize=(8, 6))
sns.barplot(x='Tipo', y='Precisión', data=df_metrics, palette='Blues_d')
plt.title('Precisión por Tipo')
plt.ylim(0, 1)
plt.ylabel('Precisión')

# Guardar la figura como un archivo PNG
plt.savefig(f'resultados/{name}/precision_{name}.png', dpi=300, bbox_inches='tight')

plt.show()

# Gráfico de barras para Recall
plt.figure(figsize=(8, 6))
sns.barplot(x='Tipo', y='Recall', data=df_metrics, palette='Greens_d')
plt.title('Recall por Tipo')
plt.ylim(0, 1)
plt.ylabel('Recall')

# Guardar la figura como un archivo PNG
plt.savefig(f'resultados/{name}/recall_{name}.png', dpi=300, bbox_inches='tight')

plt.show()

# Gráfico de barras para F1-score
plt.figure(figsize=(8, 6))
sns.barplot(x='Tipo', y='F1-score', data=df_metrics, palette='Reds_d')
plt.title('F1-score por Tipo')
plt.ylim(0, 1)
plt.ylabel('F1-score')
# Guardar la figura como un archivo PNG
plt.savefig(f'resultados/{name}/f1score_{name}.png', dpi=300, bbox_inches='tight')
plt.show()

# Gráfico combinado
df_metrics_melted = pd.melt(df_metrics, id_vars=['Tipo'], value_vars=['Precisión', 'Recall', 'F1-score'], var_name='Métrica', value_name='Valor')

plt.figure(figsize=(10, 6))
sns.barplot(x='Tipo', y='Valor', hue='Métrica', data=df_metrics_melted)
plt.title('Métricas por Tipo')
plt.ylim(0, 1)
plt.ylabel('Valor')
plt.legend(title='Métrica')

# Guardar la figura como un archivo PNG
plt.savefig(f'resultados/{name}/combinado_{name}.png', dpi=300, bbox_inches='tight')

plt.show()


In [ ]:
# Seleccionar las columnas que empiezan con "Diar_"
diar_columns = [col for col in df_resultados.columns if col.startswith('Diar_')]

# Cambiar los valores de 1 a 0.8 en esas columnas
df_resultados[diar_columns] = df_resultados[diar_columns].replace(1, 0.8)

In [ ]:
# Definir las columnas que deseas graficar
columns_to_plot = ['Elan_FEM', 'Diar_FEM']

# Definir colores contrastantes
colors = ['#FF6347', '#4682B4']  # Tomato y SteelBlue

# Crear la gráfica de dispersión para mostrar solo los estados sin transiciones
plt.figure(figsize=(14, 8))

# Graficar solo los puntos donde el valor es 1
for i, column in enumerate(columns_to_plot):
    plt.scatter(df_resultados.index, df_resultados[column], label=column, color=colors[i], s=10)

# Configuración del gráfico
plt.title('Estados de Elan_FEM y Diar_FEM')
plt.xlabel('Tiempo [seg]')
plt.ylabel('Valores Binarios')
plt.legend(loc='upper right')
plt.grid(True)
plt.xlim(0, 30000)

# Guardar la figura como un archivo PNG
# plt.savefig(f'binarios_{name}.png', dpi=300, bbox_inches='tight')
# Mostrar la gráfica
plt.show()


In [ ]:
# Definir las columnas que deseas graficar
columns_to_plot = ['Elan_FEM', 'Diar_FEM']

# Definir colores contrastantes
colors = ['#FF6347', '#4682B4']  # Tomato y SteelBlue

# Crear la gráfica de dispersión para mostrar solo los estados sin transiciones
plt.figure(figsize=(14, 8))

# Graficar solo los puntos donde el valor es 1
for i, column in enumerate(columns_to_plot):
    plt.scatter(df_resultados.index, df_resultados[column], label=column, color=colors[i], s=10)

# Configuración del gráfico
plt.title('Estados de Elan_FEM y Diar_FEM')
plt.xlabel('Tiempo [seg]')
plt.ylabel('Valores Binarios')
plt.legend(loc='upper right')
plt.grid(True)
plt.xlim(1000, 1400)

# Guardar la figura como un archivo PNG
# plt.savefig(f'binariosShort_{name}.png', dpi=300, bbox_inches='tight')

# Mostrar la gráfica
plt.show()